# Hubble Source Catalog API Notebook
### 2019 - 2022, Rick White, Trenton McKinney

A [new MAST interface][1] supports queries to the current and previous versions of the [Hubble Source Catalog][2]. It allows searches of the summary table (with multi-filter mean photometry) and the detailed table (with all the multi-epoch measurements).  It also has an associated [API][3], which is used in this notebook.

This is based on [HSC Use Case #3][4].
* It searches the HSC for variable objects in the vicinity of dwarf galaxy IC 1613,
* shows the positions of those objects in a color-magnitude diagram,
* extracts light curves for an example object, and
* displays cutout images from the Hubble observations that were used for the light curve measurements.

The whole process takes only 30 seconds to complete.

Another [notebook][5] generates a color-magnitude diagram for the Small Magellanic Cloud in only a couple of minutes.  A more complex notebook that shows how to access the proper motion tables using the HSC API is also [available][6].


  [1]: https://catalogs.mast.stsci.edu/hsc
  [2]: https://archive.stsci.edu/hst/hsc
  [3]: https://catalogs.mast.stsci.edu/docs/hsc.html
  [4]: https://archive.stsci.edu/hst/hsc/help/use_case_3_v2.html
  [5]: ../HSCV3_SMC_API/hscv3_smc_api.ipynb
  [6]: ../SWEEPS_HSCV3P1_API/sweeps_hscv3p1_api.ipynb

# Instructions: 
* Complete the initialization steps [described below](#Initialization).
* Run the notebook.

Running the notebook from top to bottom takes about 30 seconds.


# Table of Contents
* [Initialization](#Initialization)
* [Get metadata on available HSC columns](#metadata)
* [Find variable objects in IC 1613](#ic1613)
    * [Use MAST name resolver](#resolver)
    * [Search HSC summary table](#summary)
    * [Plot variability index versus magnitude](#variability)
    * [Show variable objects in a color-magnitude diagram](#cmd)
* [Get HSC light curve for a variable](#lightcurve)
* [Extract HLA cutout images for the variable](#cutouts)

# Initialization <a class="anchor" id="Initialization"></a>

### Install Python modules

1. _This notebook requires the use of **Python 3**._
1. Modules can be installed with `conda`, if using the [Anaconda distribution][1] of python, or with `pip`.
   - If you are using `conda`, do not install / update / remove a module with `pip`, that exists in a `conda` [channel][2].
   - If a module is not available with `conda`, then it's okay to install it with `pip`


  [1]: https://www.anaconda.com/products/distribution
  [2]: https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/channels.html

In [ ]:
import astropy
from astropy.coordinates import SkyCoord
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

from astropy.table import Table
import pandas as pd

from PIL import Image
from io import BytesIO, StringIO

# set width for pprint
astropy.conf.max_width = 150

In [ ]:
# set universal matplotlib parameters
plt.rcParams.update({'font.size': 16})

### MAST API functions

- Execute HSC searches and resolve names using [MAST query][1].
- Here we define several interrelated functions for retrieving information from the MAST API.
  - The `hcvcone(ra, dec, radius [, keywords])` function searches the HCV catalog near a position.
  - The `hcvsearch()` function performs general non-positional queries.
  - The `hcvmetadata()` function gives information about the columns available in a table. 
  
  
  [1]: https://mast.stsci.edu/api/v0/MastApiTutorial.html

In [ ]:
hscapiurl = "https://catalogs.mast.stsci.edu/api/v0.1/hsc"


def hsccone(ra, dec, radius, table="summary", release="v3", format="csv", magtype="magaper2",
            columns=None, baseurl=hscapiurl, verbose=False, **kw):
    """Do a cone search of the HSC catalog

    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json, table
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2)
    """

    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return hscsearch(table=table, release=release, format=format, magtype=magtype,
                     columns=columns, baseurl=baseurl, verbose=verbose, **data)


def hscsearch(table="summary", release="v3", magtype="magaper2", format="csv",
              columns=None, baseurl=hscapiurl, verbose=False, **kw):
    """Do a general search of the HSC catalog (possibly without ra/dec/radius)

    Parameters
    ----------
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json, table
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2).  Note this is required!
    """

    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    if format not in ("csv", "votable", "json", 'table'):
        raise ValueError("Bad value for format")
    if format == "table":
        rformat = "csv"
    else:
        rformat = format
    url = f"{cat2url(table, release, magtype, baseurl=baseurl)}.{rformat}"
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in hscmetadata(table, release, magtype)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError(f"Some columns not found in table: {', '.join(badcols)}")
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = f"[{','.join(columns)}]"

    # either get or post works
    # r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    elif format == "table":
        # use pandas to work around bug in Windows for ascii.read
        return Table.from_pandas(pd.read_csv(StringIO(r.text)))
    else:
        return r.text


def hscmetadata(table="summary", release="v3", magtype="magaper2", baseurl=hscapiurl):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    url = f"{cat2url(table, release, magtype, baseurl=baseurl)}/metadata"
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'], x['type'], x['description']) for x in v],
                names=('name', 'type', 'description'))
    return tab


def cat2url(table="summary", release="v3", magtype="magaper2", baseurl=hscapiurl):
    """Return URL for the specified catalog and table
    
    Parameters
    ----------
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns a string with the base URL for this request
    """
    checklegal(table, release, magtype)
    if table == "summary":
        url = f"{baseurl}/{release}/{table}/{magtype}"
    else:
        url = f"{baseurl}/{release}/{table}"
    return url


def checklegal(table, release, magtype):
    """Checks if this combination of table, release and magtype is acceptable
    
    Raises a ValueError exception if there is problem
    """
    
    releaselist = ("v2", "v3")
    if release not in releaselist:
        raise ValueError(f"Bad value for release (must be one of {', '.join(releaselist)})")
    if release == "v2":
        tablelist = ("summary", "detailed")
    else:
        tablelist = ("summary", "detailed", "propermotions", "sourcepositions")
    if table not in tablelist:
        raise ValueError(f"Bad value for table (for {release} must be one of {', '.join(tablelist)})")
    if table == "summary":
        magtypelist = ("magaper2", "magauto")
        if magtype not in magtypelist:
            raise ValueError(f"Bad value for magtype (must be one of {', '.join(magtypelist)})")

## Get metadata on available columns <a name="metadata"></a>

The `metadata` query returns information on the columns in the table.  It works for any of the tables in the API (`summary`, `detailed`, `propermotions`, `sourcepositions`).

Note that the summary table has a huge number of columns!  Each of the 133 filter/detector combinations has 3 columns with the magnitude, median absolute deviation (MAD, a robust measure of the scatter among the measurements), and the number of independent measurements in the filter.  The filter name includes a prefix for the detector (`A`=ACS/WFC, `W3`=WFC3/UVIS or WFC3/IR, `W2`=WFPC2) followed by the standard name of the filter.  So for instance all three instruments have an F814W filter, so there are columns for `A_F814W`, `W3_F814W`, and `W2_F814W`.

In [ ]:
meta = hscmetadata("summary")
print(len(meta), "columns in summary")
filterlist = meta['name'][19::3].tolist()
print(len(filterlist), "filters")
pprint(filterlist, compact=True)
meta[:19]

## Find variable objects in the dwarf irregular galaxy IC 1613 <a name="ic1613"></a>

This is based on [HSC Use Case #3](https://archive.stsci.edu/hst/hsc/help/use_case_3_v2.html), which shows an example of selecting objects from the HSC in portal.  This is simple to do using the HSC API.

### Use `astropy` name resolver to get position of IC 1613 <a name="resolver"></a>

In [ ]:
target = 'IC 1613'
coord_ic1613 = SkyCoord.from_name(target)

ra_ic1613 = coord_ic1613.ra.degree
dec_ic1613 = coord_ic1613.dec.degree
print(f'ra: {ra_ic1613}\ndec: {dec_ic1613}')

### Select objects with enough measurements to determine variability <a name="summary"></a>

This searches the summary table for objects within 0.5 degrees of the galaxy center that have at least 10 measurements in both ACS F475W and F814W.

In [ ]:
# save typing a quoted list of columns
columns = """MatchID,MatchRA,MatchDec,NumFilters,NumVisits,NumImages,StartMJD,StopMJD,
    A_F475W, A_F475W_N, A_F475W_MAD,
    A_F814W, A_F814W_N, A_F814W_MAD""".split(",")
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

constraints = {'A_F475W_N.gte': 10, 'A_F814W_N.gte': 10}

t0 = time.time()
tab = hsccone(ra_ic1613, dec_ic1613, 0.5, table="summary", release='v3', columns=columns, verbose=True, format="table", **constraints)
print(f"{(time.time()-t0):.1f} s: retrieved data and converted to {len(tab)}-row astropy table")

# clean up the output format
tab['A_F475W'].format = "{:.3f}"
tab['A_F475W_MAD'].format = "{:.3f}"
tab['A_F814W'].format = "{:.3f}"
tab['A_F814W_MAD'].format = "{:.3f}"
tab['MatchRA'].format = "{:.6f}"
tab['MatchDec'].format = "{:.6f}"
tab['StartMJD'].format = "{:.5f}"
tab['StopMJD'].format = "{:.5f}"
tab

### Plot object positions on the sky

We mark the galaxy center as well.  Note that this field is in the outskirts of IC 1613.  The 0.5 search radius (which is the maximum allowed in the API) allows finding these objects.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot('MatchRA', 'MatchDec', 'bo', markersize=1, data=tab, label=f'{len(tab)} HSC measurements')
ax.plot(ra_ic1613, dec_ic1613, 'rx', label=target, markersize=10)
ax.set(xlabel='RA [deg]', ylabel='Dec [deg]', aspect='equal')
ax.invert_xaxis()
_ = ax.legend(loc='best')

### Plot MAD variability index versus magnitude in F475W <a name="variability"></a>

The median absolute deviation is measured among the ~12 magnitude measurements in the catalog.  Some scatter is expected from noise (which increases for fainter objects).   Objects with MAD values that are high are likely to be variable.

Select variable objects that are not too faint.

In [ ]:
wvar = np.where((tab['A_F475W_MAD'] > 0.1) & (tab['A_F475W'] < 24) & (tab['A_F475W'] > 21))[0]

fig, ax = plt.subplots(figsize=(10, 10))
ax.plot('A_F475W', 'A_F475W_MAD', 'bo', markersize=2, alpha=0.1, data=tab,
        label=f'{len(tab)} HSC measurements near {target}')
ax.plot('A_F475W', 'A_F475W_MAD', 'ro', markersize=5, data=tab[wvar],
        label=f'{len(wvar)} variable candidates')
ax.set(xlabel='A_F475W [mag]', ylabel='A_F475W_MAD [mag]')
_ = ax.legend(loc='best')

### Check positions of variable objects in a color-magnitude diagram <a name="cmd"></a>

Note that these objects are generally located in the Cepheid instability strip.

In [ ]:
b_minus_i = tab['A_F475W'] - tab['A_F814W']

fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(b_minus_i, tab['A_F475W'], 'bo', markersize=2, alpha=0.1,
        label=f'{len(tab)} HSC measurements near {target}')
ax.plot(b_minus_i[wvar], tab['A_F475W'][wvar], 'ro', markersize=5,
        label=f'{len(wvar)} variable candidates')
ax.set(xlabel='A_F475W - A_F814W [mag]', ylabel='A_F475W [mag]')
ax.invert_yaxis()
_ = ax.legend(loc='best')

### Query the API for the light curve for one of the objects <a name="lightcurve"></a>

Select the most variable object as an example. 

In [ ]:
wvar = wvar[np.argsort(-tab['A_F475W_MAD'][wvar])]
iselect = wvar[0]
print(f"MatchID {tab['MatchID'][iselect]} B = {tab['A_F475W'][iselect]:.3f} B-I = {b_minus_i[iselect]:.3f}")
tab[wvar]

Get column metadata for detailed observation table (which has time-dependent magnitudes).

In [ ]:
meta = hscmetadata("detailed")
print(len(meta), "columns in detailed")
pprint(meta['name'].tolist(), compact=True)

### Get separate light curves for F475W and F814W from the detailed table

In [ ]:
columns = """MatchID,SourceID,StartMJD,Detector,Filter,MagAper2,Flags,ImageName""".split(",")
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

constraints = {'MatchID': tab['MatchID'][iselect], 'Detector': 'ACS/WFC'}
t0 = time.time()
f475 = hscsearch(table="detailed", release='v3', columns=columns, Filter='F475W', format="table", **constraints)
f814 = hscsearch(table="detailed", release='v3', columns=columns, Filter='F814W', format="table", **constraints)
print(f"{time.time()-t0:.1f} s: retrieved data and converted to {len(f475)} (F475W) and {len(f814)} (F814W) row astropy tables")

f475.sort('StartMJD')
f814.sort('StartMJD')
f475['MagAper2'].format = "{:.3f}"
f475['StartMJD'].format = "{:.5f}"
f814['MagAper2'].format = "{:.3f}"
f814['StartMJD'].format = "{:.5f}"

f475

### Plot the light curves

The light curves appear well-behaved and are closely correlated in the two filters.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), tight_layout=True)

ax.plot('StartMJD', 'MagAper2', 'bo', data=f475, label='ACS/WFC F475W')
ax.plot('StartMJD', 'MagAper2', 'ro', data=f814, label='ACS/WFC F814W')

ax.set(xlabel='MJD [days]', ylabel='[mag]')
ax.invert_yaxis()
ax.legend()

### Extract HLA cutout images for the F475W images <a name="cutouts"></a>

Get HLA F475W cutout images for the example variable.  The `get_hla_cutout` function reads a single cutout image (as a JPEG grayscale image) and returns a PIL image object.  See the documentation on the [fitscut image cutout service](http://hla.stsci.edu/fitscutcgi_interface.html) for more information on the web service being used.

Examination of the images can be useful to identified cosmic-ray contamination and other possible image artifacts.  In this case, no issues are seen, so the light curve is likely to be reliable.

In [ ]:
def get_hla_cutout(imagename, ra, dec, size=33, autoscale=99.5, asinh=1, zoom=1):
    
    """Get JPEG cutout for an image"""
    
    url = "https://hla.stsci.edu/cgi-bin/fitscut.cgi"
    r = requests.get(url, params=dict(ra=ra, dec=dec, size=size, format="jpeg",
                                      red=imagename, autoscale=autoscale, asinh=asinh, zoom=zoom))
    im = Image.open(BytesIO(r.content))
    return im

In [ ]:
# sort images by magnitude from faintest to brightest
isort = np.argsort(-f475['MagAper2'])

imagename = f475['ImageName'][isort]
mag = f475['MagAper2'][isort]
mjd = f475['StartMJD'][isort]

nim = len(imagename)
ncols = 4 # images per row
nrows = (nim+ncols-1)//ncols

imsize = 15
mra = tab['MatchRA'][iselect]
mdec = tab['MatchDec'][iselect]

In [ ]:
# download list of images; might take a minute
images = [get_hla_cutout(imagename[k], mra, mdec, size=imsize) for k in range(nim)]

In [ ]:
plt.rcParams.update({"font.size": 11})
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, (15/ncols)*nrows), tight_layout=True)

axes = axes.flat

for i, (ax, img) in enumerate(zip(axes, images)):
    ax.imshow(img, origin="upper", cmap="gray")
    ax.set_title(f'{mjd[i]:.5f} f475w={mag[i]:.3f}')